In [ ]:
from Models import * 
from pathlib import Path
import re
import numpy as np 

In [ ]:
database.create_tables([Sample, Mutations])

In [ ]:
current_directiry =  Path().absolute()

database.connect()

In [ ]:
for fn in Path(current_directiry).glob("*.vcf"): #проход по папке 
    sample_data = str(fn).split(".")
    sample_type = sample_data[1]
    sample_citonumber = int(sample_data[2].split("_")[1])
    print(sample_type, sample_citonumber)
    try:
        sample_to_base = Sample.create(Sample_name = sample_citonumber, Type = sample_type)
    except IntegrityError:
            continue
    f = open(fn, 'r')
    for l in f :
        if (not l.startswith('##')) and (not l.startswith('#')):
            mutation_data = l.split('\t')
            #print(mutation_data)
            chromosome_data = mutation_data[0]
            position_data  =  mutation_data[1]
            mutation_name_data  = mutation_data[2]
            read_depth_data= int(re.search(r'DP=(\d+)', mutation_data[7] )[1]) #? 
            alleles = np.array( mutation_data[-1].split(':')[0].split('/'),int  )
            #types = {0: mutation_data[3] }
            types = [mutation_data[3]]
            for i in range(alleles.max()):
                types.append(mutation_data[4][i])
   
            genotypes_data = types[alleles[0] ] + '/' + types[alleles[1] ]
            #print(genotypes_data)
            mutation_to_base = Mutations.create( sample = sample_to_base,chromosome = chromosome_data,position =  position_data,
                                        mutation_name = mutation_name_data,read_depth = read_depth_data,genotype = genotypes_data )

            print(chromosome_data, position_data)
    
    f.close()
 

In [ ]:
for sample in Sample.select():
    #Sample.get(Sample.Sample_name == sample.Sample_name).delete_instance()
    print(sample.Sample_name, sample.Type)

In [ ]:
database.close()